# Fermat distance - Coefficient of Variation

Add table + introduction

In [4]:
from fermat.L_fermat_stat import *
import concurrent.futures as cf
import logging
from tqdm import tqdm

# Cylinder

# GOOD CODE Multiple simulations (5/10/22)

In [5]:
np.linspace(1.5, 8., 14, endpoint=True)

array([1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. , 7.5,
       8. ])

# No Harmonizer

In [6]:
#Parameters

all_n = [50] #Sample size
all_d = [1]
#all_d = [1,2,3,4,5]
#all_d = [1]
#all_n = [100, 200, 400, 800, 1600]
all_alpha = np.linspace(1.5, 8., 14, endpoint=True) #Fermat dist parameter

n_simulations = 50 #Number of experiences
n_batch = 25 
all_n_simulations = np.repeat(n_batch, n_simulations/n_batch)
assert np.sum(all_n_simulations) == n_simulations

#Harmonizer or Not
h = 0
harmonizer = [None,'Knn','Epsilon_radius','Delaunay'][h]
n_harmonizers = 1
k = None
eps = None

n_workers = 10

import itertools
all_n, all_d, all_alpha, all_n_simulations = zip(*itertools.product(all_n, all_d, all_alpha, all_n_simulations))

# we shuffle to help parallelization
from sklearn.utils import shuffle
all_n, all_d, all_alpha, all_n_simulations = shuffle(all_n, all_d, all_alpha, all_n_simulations, random_state=0)

# since each worker works as a separate machine, we need to somehow fix the seed for each taks
#n_tasks = int(n_simulations/n_batch)
#all_seed = np.random.randint(0, 2**32-1, n_tasks)
#all_seed = range(int(n_simulations/n_batch))

n_tasks = len(all_alpha)
all_seed = np.random.randint(0, 2**32-1, n_tasks)
assert len(np.unique(all_seed)) == n_tasks

title_csv = 'xp_result/Fermat_CV_Cylinder_NoHarmonizer_d1to5.csv'

In [7]:
%%time
with cf.ProcessPoolExecutor(max_workers=n_workers) as executor:  
    future_to_ind = {executor.submit(fermat_simu_cylinder, n, d, [a], 1, b, harmonizer, n_harmonizers, k, eps, s): ind 
                     for ind, (n,d,a,b,s) in enumerate(zip(all_n,all_d,all_alpha,all_n_simulations,all_seed))}
    logging.info("Submitted_futures")
    results=[]
    for future in tqdm(cf.as_completed(future_to_ind)):
        ind = future_to_ind[future]
        try:
            result_N_dict = future.result()
            results.append(result_N_dict)
        except Exception as exc:
            logging.error(f"{ind} generated an exception: {exc}", exc_info=True)
df = pd.concat(results)
#df.to_csv(title_csv)

28it [00:03,  8.40it/s]


CPU times: user 64 ms, sys: 97.4 ms, total: 161 ms
Wall time: 3.96 s


# Knn-Harmonizer

In [13]:
#Parameters

#all_n = [50] #Sample size
#all_d = [1]
all_d = [1,2,3,4,5]
#all_d = [1]
all_n = [100, 200, 400, 800, 1600]
all_alpha = np.linspace(1.5, 8., 14, endpoint=True) #Fermat dist parameter

n_simulations = 50 #Number of experiences
n_batch = 25 
all_n_simulations = np.repeat(n_batch, n_simulations/n_batch)
assert np.sum(all_n_simulations) == n_simulations

#Harmonizer or Not
h = 1
harmonizer = [None,'Knn','Epsilon_radius','Delaunay'][h]
n_harmonizers = 1
all_k = np.arange(2,6)
eps = None

n_workers = 10

import itertools
all_n, all_d, all_alpha, all_n_simulations, all_k = zip(*itertools.product(all_n, all_d, all_alpha, all_n_simulations, all_k))

# we shuffle to help parallelization
from sklearn.utils import shuffle
all_n, all_d, all_alpha, all_n_simulations, all_k = shuffle(all_n, all_d, all_alpha, all_n_simulations, all_k, random_state=0)

# since each worker works as a separate machine, we need to somehow fix the seed for each taks
#n_tasks = int(n_simulations/n_batch)
#all_seed = np.random.randint(0, 2**32-1, n_tasks)
#all_seed = range(int(n_simulations/n_batch))

n_tasks = len(all_alpha)
all_seed = np.random.randint(0, 2**32-1, n_tasks)
assert len(np.unique(all_seed)) == n_tasks

title_csv = 'xp_result/Fermat_CV_Cylinder_Knn2to5_d1to5.csv'

In [14]:
%%time
with cf.ProcessPoolExecutor(max_workers=n_workers) as executor:  
    future_to_ind = {executor.submit(fermat_simu_cylinder, n, d, [a], 1, b, harmonizer, n_harmonizers, k, eps, s): ind 
                     for ind, (n,d,a,b,k,s) in enumerate(zip(all_n,all_d,all_alpha,all_n_simulations,all_k,all_seed))}
    logging.info("Submitted_futures")
    results=[]
    for future in tqdm(cf.as_completed(future_to_ind)):
        ind = future_to_ind[future]
        try:
            result_N_dict = future.result()
            results.append(result_N_dict)
        except Exception as exc:
            logging.error(f"{ind} generated an exception: {exc}", exc_info=True)
df = pd.concat(results)
#df.to_csv(title_csv)

112it [00:04, 26.93it/s]


CPU times: user 150 ms, sys: 102 ms, total: 253 ms
Wall time: 4.83 s
